    16_230104_003_adress_transform
    
    data .. 입력

In [5]:
# 필요한 라이브러리 import 

import requests
import pprint
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

print("슝~")

슝~


In [6]:
import requests

### SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
  
    return access_token

# 여러분의 서비스ID, 보안Key 값을 함수에 넣어서 잘 작동하는지 확인해보세요
consumer_key = "2f74504a63224ee09f2b"   # 서비스ID
consumer_secret = "63a0ffef893e43a48ae2" # 보안Key

# get_access_token('서비스ID', '보안Key')
print('access token: ', get_access_token(consumer_key, consumer_secret))

access token:  317f84cd-4ecf-4fea-a2cf-5f030cd017bf


In [8]:
#4. code 변환
## SGIS access_token 가져오기
access_token = get_access_token(consumer_key, consumer_secret)

## 주소를 입력해 주소의 데이터를 DataFrame으로 가지고오는 함수
def get_adress_info(access_token, address) -> None:
    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    temp_df = pd.DataFrame(temp_res)

    return temp_df

print("슝~")

슝~


In [9]:
# 예시) 인천공항 주소 정보 DataFrame으로 가지고 오기
# 인천공항 주소: 인천 중구 공항로 271
address_df = get_adress_info(access_token, "인천 중구 공항로 271")
address_df

,ri_nm,road_nm_sub_no,sgg_cd,adm_cd,road_nm_main_no,leg_cd,road_nm,bd_matches,bd_sub_nm,addr_type,...,road_cd,bd_main_nm,adm_nm,jibun_main_no,origin_xy,jibun_sub_no,ri_cd,y,leg_nm,x
0,null,0,23010,23010640,271,2811014700,공항로,4,null,6,...,3149003,인천국제공항역,운서동,null,BUILD,null,2811014700,1939236.69257386,운서동,907316.512763601


In [10]:
## 위도(x), 경도(y)의 데이터 타입을 float으로 변경하기
address_df["x"] = address_df["x"].astype(float)
address_df["y"] = address_df["y"].astype(float)

address_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ri_nm            1 non-null      object 
 1   road_nm_sub_no   1 non-null      object 
 2   sgg_cd           1 non-null      object 
 3   adm_cd           1 non-null      object 
 4   road_nm_main_no  1 non-null      object 
 5   leg_cd           1 non-null      object 
 6   road_nm          1 non-null      object 
 7   bd_matches       1 non-null      object 
 8   bd_sub_nm        1 non-null      object 
 9   addr_type        1 non-null      object 
 10  sido_nm          1 non-null      object 
 11  sgg_nm           1 non-null      object 
 12  sido_cd          1 non-null      object 
 13  road_cd          1 non-null      object 
 14  bd_main_nm       1 non-null      object 
 15  adm_nm           1 non-null      object 
 16  jibun_main_no    1 non-null      object 
 17  origin_xy        1 n

In [11]:
## 주소정보중 위/경도 데이터로 Geometry Point 객체를 생성해서 Geometry 컬럼 생성하기
address_df["geometry"] = address_df.apply(lambda row: Point([row["x"], row["y"]]), axis=1)

## Geopandas DataFrame으로 DataFrame 생성하기
address_geo_df = gpd.GeoDataFrame(address_df, geometry="geometry")
address_geo_df[["bd_main_nm", "x", "y", "geometry"]]

,bd_main_nm,x,y,geometry
0,인천국제공항역,907316.512764,1.939237e+06,POINT (907316.513 1939236.693)


In [12]:
## 좌표계 확인
# None 값인 경우 좌표계를 정의 해주어야 합니다.
print(address_geo_df.crs)

None


In [13]:
## 위/경도 좌표계 정의
# SGIS 기본 좌표계 UTM-K 좌표계를 사용합니다.
# UTM-K 좌표계의 EPSG코드는 epsg:5179를 사용합니다.
address_geo_df.crs = {"init": "epsg:5179"}
address_geo_df.crs

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


<Projected CRS: +init=epsg:5179 +type=crs>
Name: Korea 2000 / Unified CS
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Republic of Korea (South Korea) - onshore and offshore.
- bounds: (122.71, 28.6, 134.28, 40.27)
Coordinate Operation:
- name: Korea Unified Belt
- method: Transverse Mercator
Datum: Geocentric datum of Korea
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [14]:
## UTM-K 좌표계 -> WGS84(위/경도) 좌표계 변환
address_geo_df = address_geo_df.to_crs({"init": "epsg:4326"})
address_geo_df[["bd_main_nm", "x", "y", "geometry"]]

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,bd_main_nm,x,y,geometry
0,인천국제공항역,907316.512764,1.939237e+06,POINT (126.45217 37.44768)


In [15]:
# 예시) 국립 현충원 주소 정보 DataFrame으로 가지고 오기
# 국립 현충원 주소: 서울특별시 동작구 동작동 271-18
address_df = get_adress_info(access_token, "서울특별시 동작구 동작동 271-18")
address_df[["sido_nm", "sgg_nm", "adm_nm", "jibun_main_no", "jibun_sub_no", "x", "y"]]

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y
0,서울특별시,동작구,사당2동,271,18,953475.620000162,1944631.15909765


In [16]:
### UTM-K 좌표계를 WGS84 좌표계로 변환하는 함수
# 좌표계별 EPSG CODE 종류: UTM-K 좌표계(EPSG:5179), WGS84(위/경도 좌표계, EPSG:4326)

def utmk_to_wgs84(utmk_x, utmk_y) -> None:
    utmk_proj = pyproj.Proj(init="EPSG:5179") # UTM-K 좌표계 정의
    wgs84_proj = pyproj.Proj(init="EPSG:4326") # WGS84 좌표계 정의
    
    # 좌표 변환 수헹
    wgs84_x, wgs84_y = pyproj.transform(utmk_proj, wgs84_proj, utmk_x, utmk_y)
    # print(f"UTM-K 위도: {utmk_y} -> WGS84 위도: {wgs84_y} \nUTM-K 경도: {utmk_x} -> WGS84 경도: {wgs84_x}")
    
    
    return wgs84_x, wgs84_y

print("슝~")

슝~


In [17]:
# 변환된 좌푯값을 가져와 데이터 프레임에 컬럼으로 추가하기
address_df[["wgs84_x", "wgs84_y"]] = utmk_to_wgs84(address_df.loc[0, "x"], address_df.loc[0, "y"])
address_df[["sido_nm", "sgg_nm", "adm_nm", "jibun_main_no", "jibun_sub_no", "x", "y", "wgs84_x", "wgs84_y"]]

/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/conda/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_111/3540108372.py:9: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  wgs84_x, wgs84_y = pyproj.transform(utmk_proj, 

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y,wgs84_x,wgs84_y
0,서울특별시,동작구,사당2동,271,18,953475.620000162,1944631.15909765,126.973651,37.499772


In [18]:
#SGIS API를 활용한 좌표계 변환
# 예시) 서울시청 주소 정보 DataFrame으로 가지고 오기
# 서울시청 주소: 서울 중구 세종대로 110 서울특별시청
address_df = get_adress_info(access_token, "서울 중구 세종대로 110 서울특별시청")
address_df[["sido_nm", "sgg_nm", "adm_nm", "jibun_main_no", "jibun_sub_no", "x", "y"]]

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y
0,서울특별시,중구,명동,null,null,953893.87330747,1952017.73064447


In [19]:
## SGIS 좌표변환 API를 활용한 좌표변환 함수 정의
def get_transcoord(access_token, src, dst, posX, posY) -> None:
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌푯값 딕셔너리의 value 리턴

print("슝~")

슝~


In [20]:
# 변환후 데이터 프레임에 변환된 좌표 컬럼 추가하기
address_df[["transcoord_x", "transcoord_y"]] = get_transcoord(access_token, "EPSG:5179", "EPSG:4326", address_df.loc[0, "x"], address_df.loc[0, "y"])
address_df[["sido_nm", "sgg_nm", "adm_nm", "jibun_main_no", "jibun_sub_no", "x", "y", "transcoord_x", "transcoord_y"]]

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y,transcoord_x,transcoord_y
0,서울특별시,중구,명동,null,null,953893.87330747,1952017.73064447,126.977918,37.56637


In [21]:
# 예시) 제주공항 주소 정보 DataFrame으로 가지고 오기
# 제주공항 주소: 제주 제주시 공항로 2 제주국제공항
address_df = get_adress_info(access_token,  "제주 제주시 공항로 2 제주국제공항") # 주소입력
address_df[["sido_nm", "sgg_nm", "adm_nm", "jibun_main_no", "jibun_sub_no", "x", "y"]]

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y
0,제주특별자치도,제주시,용담2동,null,null,906497.939648858,1502114.73637921


In [22]:
# 변환후 데이터 프레임에 변환된 좌표 컬럼 추가하기
address_df[["transcoord_x", "transcoord_y"]] = get_transcoord(access_token, "EPSG:5179", "EPSG:4326", address_df.loc[0, "x"], address_df.loc[0, "y"])  
address_df[["sido_nm", "sgg_nm", "adm_nm", "jibun_main_no", "jibun_sub_no", "x", "y", "transcoord_x", "transcoord_y"]]

,sido_nm,sgg_nm,adm_nm,jibun_main_no,jibun_sub_no,x,y,transcoord_x,transcoord_y
0,제주특별자치도,제주시,용담2동,null,null,906497.939648858,1502114.73637921,126.493302,33.506828
